In [37]:
import sys
!{sys.executable} -m pip install astropy
!{sys.executable} -m pip install sep
import numpy as np
import sep

downloads neccesary packages and makes sure that they are initialized by python

In [21]:
import astropy.io.fits as fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

imports the rest of the neccesary packages and running configuration parameters

In [22]:
data = fits.getdata("image.fits")

gets the data out of image.fits to analyze with the following functions

In [23]:
data = data.byteswap().newbyteorder()
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray',vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig("fitsfilep1.png");

fits from astropy wants to look at the data in a decending order so we have to swap the bit order for it to look at the data like fitsio would
sets m equal to the numpy function for mean
sets s equal to the numpy function for standard deviation
shows the image with given parameters and outputs a bar next to it with the given range
its not showing an output in the code unless I save it so  i had to save it as a png

In [24]:
data = data.byteswap().newbyteorder()
bkg = sep.Background(data)

had to run byteswap again to prevent an error
setting a variable for the background

In [25]:
print(bkg.globalback)
print(bkg.globalrms)

6852.04931640625
65.46174621582031


prints the total mean of the background value and the noise of the background value

In [26]:
bkg_image = bkg.back()

looks at the background as a 2d array

In [27]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig("fitsfilep2.png")

shows the background

In [28]:
bkg_rms = bkg.rms()

looks at the backgtound noise as a 2d array

In [29]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig("fitsfilep3.png")

shows the background noise

In [30]:
data_sub = data - bkg

subtracts the background from the image as a whole

In [31]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

setting the noise threshold to mark anything above 1.5

In [32]:
len(objects)

68

Shows how many objects were detected

In [33]:
from matplotlib.patches import Ellipse


fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')


for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

plt.savefig("fitsfilep4.png")

the function at the top plots the image with no background and the second function surrounds each of the identified objects with an ellipse

In [34]:
objects.dtype.names

('thresh',
 'npix',
 'tnpix',
 'xmin',
 'xmax',
 'ymin',
 'ymax',
 'x',
 'y',
 'x2',
 'y2',
 'xy',
 'errx2',
 'erry2',
 'errxy',
 'a',
 'b',
 'theta',
 'cxx',
 'cyy',
 'cxy',
 'cflux',
 'flux',
 'cpeak',
 'peak',
 'xcpeak',
 'ycpeak',
 'xpeak',
 'ypeak',
 'flag')

shows the number of available fields

In [35]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

checks how much light is coming from each object based on light, errors in the light, and accounting for noise

In [38]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

object 0: flux = 2249.159297 +/- 291.027802
object 1: flux = 3092.220430 +/- 291.592204
object 2: flux = 5949.868379 +/- 356.562003
object 3: flux = 1851.426582 +/- 295.028816
object 4: flux = 72736.386914 +/- 440.172206
object 5: flux = 3860.756152 +/- 352.163162
object 6: flux = 6418.913789 +/- 357.458973
object 7: flux = 2210.707656 +/- 350.791223
object 8: flux = 2741.607227 +/- 352.277746
object 9: flux = 20916.875566 +/- 376.966138


Shows the results of the first ten objects